In [1]:
import numpy as np
import sklearn
import pickle
from sklearn.model_selection import StratifiedKFold
from scipy.stats import norm

from data_generation import m_0, g_0, get_data
#from dml_algorithm import mm_ate, dml_ate

In [2]:
def mc_simulation(N, model_g, model_m, n_MC=5000):
    np.random.seed(100)
    ate_estimates = np.empty((n_MC, 4))
    sigma_estimates = np.empty(n_MC)
    CIs = np.empty((n_MC, 2))

    for j in range(n_MC):
        y_data, d_data, x_data = get_data(N)
        ate_estimates[j, 0] = mm_ate(y_data, d_data, x_data, g_0, m_0)
        ate_estimates[j, 1:], sigma_estimates[j], CIs[j] = dml_ate(y_data, d_data, x_data, model_g, model_m)

    return [ate_estimates, sigma_estimates, CIs]

In [15]:
def dml_ate(y_data, d_data, x_data, model_g, model_m, K=5, classical=True, inference=True, alpha=0.05):
    # Generate random partition of data for cross-fitting
    N = len(y_data)
    skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

    # Compute respective ML estimators and thereupon auxiliary estimators
    theta_0_check_list = []
    if classical:
        reg_check_list, ipw_check_list = [], []
    if inference:
        scores_list = []
    
    for (train_indices, eval_indices) in skf.split(X=x_data, y=d_data):
        y_train, d_train, x_train = y_data[train_indices], d_data[train_indices], x_data[train_indices]
        y_eval, d_eval, x_eval = y_data[eval_indices], d_data[eval_indices], x_data[eval_indices]

        # Estimate outcome regression functions g_0(d)
        g_0_hat = []
        model_g0 = Sequential()
        model_g0.add(Input(shape=(3,)))
        model_g0.add(Dense(10, activation='relu'))
        model_g0.add(Dropout(0.2))
        model_g0.add(Dense(1))
        model_g0.compile(loss='mean_squared_error', optimizer='adam')
        model_g1 = Sequential()
        model_g1.add(Input(shape=(3,)))
        model_g1.add(Dense(10, activation='relu'))
        model_g1.add(Dropout(0.2))
        model_g1.add(Dense(1))
        model_g1.compile(loss='mean_squared_error', optimizer='adam')
        model_g = [model_g0, model_g1]
        for d in [0, 1]:
            model_g[d].fit(x_train[d_train==d], y_train[d_train==d], epochs=35, batch_size=8, verbose=0)#
            g_0_hat.append(model_g[d].predict(x_eval))

        # Estimate propensity score m_0
        model_m = Sequential()
        model_m.add(Input(shape=(3,)))
        model_m.add(Dense(5, activation='relu'))
        model_m.add(Dropout(0.2))
        model_m.add(Dense(1, activation='sigmoid'))
        model_m.compile(loss='binary_crossentropy', optimizer='adam')
        model_m.fit(x_train, d_train, epochs=35, batch_size=8, verbose=0)#
        m_0_hat = model_m.predict(x_eval)#
            
        # Compute auxiliary estimator
        scores = g_0_hat[1] - g_0_hat[0] + d_eval*(y_eval-g_0_hat[1])/m_0_hat - (1-d_eval)*(y_eval-g_0_hat[0])/(1-m_0_hat)
        theta_0_check_list.append(np.mean(scores))

        # For variance estimation
        if inference:
            scores_list.append(scores)

        # For regression & IPW estimators
        if classical:
            reg_check_list.append(np.mean(g_0_hat[1] - g_0_hat[0])) 
            ipw_check_list.append(np.mean(d_eval*y_eval/m_0_hat - (1-d_eval)*y_eval/(1-m_0_hat)))     

    # Compute final estimator
    theta_0_hat = np.mean(theta_0_check_list)
    if classical:
        reg_hat, ipw_hat = np.mean(reg_check_list), np.mean(ipw_check_list)

    # Inference: estimate variance and construct confidence interval
    if inference:
        sigma_hat = np.sqrt(np.mean((np.array(scores_list)-theta_0_hat)**2))
        quantile = norm.ppf(1-alpha/2)
        CI = np.array([theta_0_hat-quantile*sigma_hat/np.sqrt(N), theta_0_hat+quantile*sigma_hat/np.sqrt(N)])

    # Return results
    if classical:
        if inference:
            return np.array([theta_0_hat, reg_hat, ipw_hat]), sigma_hat, CI
        else:
            return np.array([theta_0_hat, reg_hat, ipw_hat])
    else:
        if inference:
            return theta_0_hat, sigma_hat, CI
        else:
            return theta_0_hat

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input

In [8]:
np.random.seed(123)
N = 2000
y_data, d_data, x_data = get_data(N)

In [7]:
model_g0 = Sequential()
model_g0.add(Input(shape=(3,)))
model_g0.add(Dense(10, activation='relu'))
model_g0.add(Dropout(0.2))
model_g0.add(Dense(1))
model_g0.compile(loss='mean_squared_error', optimizer='adam')
model_g0.fit(x_data[d_data==0], y_data[d_data==0], epochs=35, batch_size=8, validation_split=0.2)

Epoch 1/35
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.7985 - val_loss: 0.7331
Epoch 2/35
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7816 - val_loss: 0.7026
Epoch 3/35
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7442 - val_loss: 0.6804
Epoch 4/35
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6916 - val_loss: 0.6590
Epoch 5/35
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7255 - val_loss: 0.6419
Epoch 6/35
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6255 - val_loss: 0.6155
Epoch 7/35
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6840 - val_loss: 0.5996
Epoch 8/35
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5987 - val_loss: 0.5796
Epoch 9/35
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6136 - val_loss: 0.5584
Epoch 10/35
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5554 - val_loss: 0.5349
Epoch 11/35
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5888 - val_loss: 0.5181
Epoch 12/35
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5362 - val_lo

In [8]:
model_g1 = Sequential()
model_g1.add(Input(shape=(3,)))
model_g1.add(Dense(10, activation='relu'))
model_g1.add(Dropout(0.2))
model_g1.add(Dense(1))
model_g1.compile(loss='mean_squared_error', optimizer='adam')
model_g1.fit(x_data[d_data==1], y_data[d_data==1], epochs=35, batch_size=8, validation_split=0.2)

Epoch 1/35
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.4194 - val_loss: 0.8790
Epoch 2/35
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8454 - val_loss: 0.6625
Epoch 3/35
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7367 - val_loss: 0.5882
Epoch 4/35
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8371 - val_loss: 0.5357
Epoch 5/35
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6814 - val_loss: 0.5239
Epoch 6/35
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5479 - val_loss: 0.5173
Epoch 7/35
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6155 - val_loss: 0.4979
Epoch 8/35
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5799 - val_loss: 0.4958
Epoch 9/35
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5576 - val_loss: 0.4913
Epoch 10/35
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5569 - val_loss: 0.4821
Epoch 11/35
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5106 - val_loss: 0.4946
Epoch 12/35
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

In [10]:
model_g0 = Sequential()
model_g0.add(Input(shape=(3,)))
model_g0.add(Dense(10, activation='relu'))
model_g0.add(Dropout(0.2))
model_g0.add(Dense(1))
model_g0.compile(loss='mean_squared_error', optimizer='adam')

model_g1 = Sequential()
model_g1.add(Input(shape=(3,)))
model_g1.add(Dense(10, activation='relu'))
model_g1.add(Dropout(0.2))
model_g1.add(Dense(1))
model_g1.compile(loss='mean_squared_error', optimizer='adam')

model_g = [model_g0, model_g1]

In [54]:
model_m = Sequential()
model_m.add(Input(shape=(3,)))
model_m.add(Dense(5, activation='relu'))
model_m.add(Dropout(0.2))
model_m.add(Dense(1, activation='sigmoid'))
model_m.compile(loss='binary_crossentropy', optimizer='adam')
model_m.fit(x_data, d_data, epochs=35, batch_size=8, validation_split=0.2)

Epoch 1/35
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8400 - val_loss: 0.6688
Epoch 2/35
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6476 - val_loss: 0.5864
Epoch 3/35
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6115 - val_loss: 0.5623
Epoch 4/35
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5853 - val_loss: 0.5500
Epoch 5/35
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5958 - val_loss: 0.5443
Epoch 6/35
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5852 - val_loss: 0.5415
Epoch 7/35
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5899 - val_loss: 0.5388
Epoch 8/35
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5827 - val_loss: 0.5351
Epoch 9/35
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5723 - val_loss: 0.5326
Epoch 10/35
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5642 - val_loss: 0.5312
Epoch 11/35
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5608 - val_loss: 0.5292
Epoch 12/35
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

In [11]:
model_m = Sequential()
model_m.add(Input(shape=(3,)))
model_m.add(Dense(5, activation='relu'))
model_m.add(Dropout(0.2))
model_m.add(Dense(1, activation='sigmoid'))
model_m.compile(loss='binary_crossentropy', optimizer='adam')

In [64]:
model_m.predict(x_data)[-15:].flatten()

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([0.7701746 , 0.78597087, 0.8578129 , 0.9682587 , 0.44352666,
       0.78307426, 0.9342824 , 0.6365525 , 0.6002639 , 0.7358402 ,
       0.14644738, 0.78670967, 0.25117138, 0.7478894 , 0.34183916],
      dtype=float32)

In [65]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_data, d_data)
logreg.predict_proba(x_data)[-15:, 1]

array([0.78148794, 0.80765035, 0.88396065, 0.97975005, 0.42626086,
       0.80256674, 0.95173849, 0.62505032, 0.60677442, 0.74982268,
       0.10918031, 0.81506017, 0.21515688, 0.77405298, 0.33681074])

In [67]:
m_0(x_data)[-15:]

array([0.80961937, 0.80817676, 0.91257872, 0.98248129, 0.4052997 ,
       0.81438695, 0.95669844, 0.61823919, 0.60095135, 0.72017885,
       0.09434526, 0.8405147 , 0.23682696, 0.76380239, 0.38884595])

In [68]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(objective='binary:logistic', max_depth=3, subsample=0.8, learning_rate=0.06, reg_lambda=10)
model_xgb.fit(x_data, d_data)
model_xgb.predict_proba(x_data)[-15:, 1]

array([0.8055433 , 0.8412702 , 0.87107456, 0.9424594 , 0.36030835,
       0.7720746 , 0.9265446 , 0.61318547, 0.71301585, 0.7792926 ,
       0.07776507, 0.8081694 , 0.27150142, 0.7862124 , 0.27370057],
      dtype=float32)

In [16]:
%%time
ate_estimates, sigma_hat, CI = dml_ate(y_data, d_data, x_data, model_g, model_m, K=2)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
CPU times: total: 28.6 s
Wall time: 20.8 s


In [17]:
ate_estimates

array([1.00078   , 1.08489418, 0.99473899])

In [23]:
CI

array([2.078498 , 2.6367546])

In [ ]:
results_dict = {}

for N, xgb_params_dict in xgb_params_dict_dict.items():
    model_g0, model_g1 = xgb.XGBRegressor(objective='reg:squarederror'), xgb.XGBRegressor(objective='reg:squarederror')
    model_g0.set_params(**xgb_params_dict['g0'])
    model_g1.set_params(**xgb_params_dict['g1'])
    model_g = [model_g0, model_g1]
    model_m = xgb.XGBClassifier(objective='binary:logistic')
    model_m.set_params(**xgb_params_dict['m'])
    results_dict[N] = mc_simulation(N, model_g, model_m)
    print(f'MC simulation done for N={N}')

with open('results_dict.pkl', 'wb') as pickle_file:
    pickle.dump(results_dict, pickle_file)